In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv

In [ ]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/")
csv_files = []
myfile = "share_by_occupation_female.txt"
# text file where I specify which files I want to use
with open(myfile, "r") as f:
    lines = f.readlines()
    for l in lines:    
        file = l.rstrip("\n") + ".csv"
        csv_files.append(file)
print(csv_files)

In [ ]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# print(result)
# Narrow down to just the files I want
result2 = [f for f in result if f in csv_files]
dfs = []
for f in result2:
    df = pd.read_csv(f)
    dfs.append((f, df))    
print(f"Using {len(dfs)} csv files")

In [ ]:
# df = dfs[0][1]
# # df.head()
# # df.tail(10)
# # ## gets rid of text dscrip at bottom
# # df_us = df_us.iloc[0:121, :]
# # # df_us.tail()
# col_nulls = df.isna().sum().sort_values()
# c = col_nulls.to_frame()
# c.reset_index(inplace=True)
# c.columns = ['Country', 'Num Nulls']
# print(c.iloc[0:4])
# # ## pick a year with low amonut of nans
# # ## e.g. 2012 

In [ ]:
# # Which year has least amount of missing data?
# for tup in dfs:
#     print(tup[0])
#     df_us = tup[1]
#     df_us = df_us.iloc[0:121, :]
#     col_nulls = df_us.isna().sum().sort_values()
#     c = col_nulls.to_frame()
#     c.reset_index(inplace=True)
#     c.columns = ['Country', 'Num Nulls']
#     print(c.iloc[0:4])
#     ## pick a year with low amonut of nans
#     ## e.g. 2012 seems like year that all DFs have low nans


## Let's narrow down to a chosen year

In [ ]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [ ]:
## NEW: map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
    # print(country_names)

country_names_dict = { v: k for k, v in country_names.items()}
# country_names_dict


In [ ]:
def convert_iso_to_name(iso):
    # print(x)
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso

In [ ]:

year = '2012'  # chosen year to graph

json_names = []
dfs_to_plot = []

for i in range(len(dfs)):
    # as a test, use first df for now
    name = dfs[i][0].strip(".csv")
    df = dfs[i][1]
    # df = df.iloc[0:121, :] # TODO: better way to get rid of bottom text?
    df = df[['ISO', year]]

    # drop nans
    df.dropna(inplace=True)

    df.columns = ['name', 'value']
    

    df['name'] = df['name'].apply(lambda x: convert_iso_to_name(x))

    dfs_to_plot.append((name, df))

    df.to_json(f"{name}.json", orient='records')
    print(f"Exported {name} data with shape {df.shape}")
    json_names.append(f"{name}.json")


In [ ]:
name = dfs_to_plot[0][0]
df = dfs_to_plot[0][1]
# print(name)

df.columns = ['name', 'value']
df.head()

In [ ]:
print(name)
plt.figure(figsize=(10,5))
plt.title("Percent of Female Managers, by Country")
plt.xticks(rotation = 45)
sns.barplot(data=df, x='name', y='value')

# Ideal nested JSON format as final product
That we can plug right into the zoomable circle packing chart 
Let names = country or code code, values = percentage 

```
{
    "name": "education",
    "children": [
        {
            "name": "Primary School Education, Women age 25+",
            "children": [
                {
                    "value": 95,
                    "name": "USA"
                },
                ...
            ]
        },
        {
            "name": "Secondary School Education, Women age 25+",
            "children": [
                {
                    "value": 96,
                    "name": "USA"
                },
                ...
            ]
        },
        ...
    ]
}

```

In [ ]:
## continue where i left off

In [ ]:
# Read back in the json files saved
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")
extension = 'json'
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
result2 = [f for f in result if f in json_names]
# print(len(result2))

In [ ]:
# import csv file with full dataset name mappings
with open('data_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dataset_names = {rows[0]: rows[1] for rows in reader}
    # print(dataset_names)


In [ ]:
# Create big json file

export_file = { "name": "employment", "children": [] }

for f in result2:
    ds_code = f.split('.json')[0]
    ds_name = dataset_names[ds_code]
    # print(ds_name)
    with open(f, "r") as read_file:
        data = json.load(read_file)  # list of dicts
        inner_dict = { "name": ds_name, "children": data }
        export_file["children"].append(inner_dict)
        


In [ ]:
with open("female_employment_data.json", "w") as write_file:
    json.dump(export_file, write_file)